In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
import os

# List contents of the current working directory
print("Contents of the current working directory:")
os.system('ls -lh /kaggle/working')


Contents of the current working directory:
total 8.0K
drwxr-xr-x 5 root root 4.0K May 11 02:18 saved-models-flan-t5-base
drwxr-xr-x 3 root root 4.0K May 11 02:18 wandb


0

In [3]:
import shutil
shutil.rmtree('/kaggle/working', ignore_errors=True)

In [4]:
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [5]:
!nvidia-smi

Sun May 11 02:20:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [7]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


In [8]:
import wandb
import os

# Login using Kaggle secret
wandb_api_key = os.environ.get('WANDB_API_KEY', None)
if wandb_api_key is None:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

wandb.login(key=wandb_api_key)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: syedmatherh789 (syedmatherh789-habib-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
import torch
import os
import evaluate
import pandas as pd
import numpy as np

if torch.cuda.is_available():
    print("GPU is enabled.")
    print("device count: {}, current device: {}".format(torch.cuda.device_count(), torch.cuda.current_device()))
else:
    print("GPU is not enabled.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #make sure GPU is enabled.

import accelerate
import transformers
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments
from torch.utils.data import DataLoader
from transformers import Seq2SeqTrainer

from nltk.tokenize import RegexpTokenizer
from torch.utils.data import DataLoader

print(transformers.__version__) #4.47.1
print(accelerate.__version__) #1.2.1

2025-05-11 02:21:26.001989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746930086.204398      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746930086.264136      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU is enabled.
device count: 2, current device: 0
4.51.1
1.3.0


## Load Model and Tokenzier

In [11]:
# model_checkpoint = 'csebuetnlp/mT5_multilingual_XLSum'
model_checkpoint="google/flan-t5-base"
model_code = model_checkpoint.split("/")[-1]
metric = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config = AutoConfig.from_pretrained(
    model_checkpoint,
    max_length=128,
    length_penalty=0.6,
    no_repeat_ngram_size=2,
    num_beams=15,
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, config=config).to(device)

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    return_tensors="pt")

print(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

google/flan-t5-base


## Prepare Data

In [13]:
!pip install langdetect deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.0 MB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=48be275eb6f2996bb860133ec940add7cfb99120bea092af31b6f1d97700db6d
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [14]:
import re
from langdetect import detect, DetectorFactory
from deep_translator import GoogleTranslator

# Set seed for reproducibility in langdetect
DetectorFactory.seed = 0

# Data preprocessing with cleaning
def preprocess_data(df):
    def clean_text(text):
        if pd.isna(text):
            return ""
        # Remove URLs and hashtags, retain some context for language detection
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'#\w+', '', text)
        text = text.strip()
        return text

    df['post'] = df['post'].apply(clean_text)
    if 'normalized_claim' in df.columns:
        df['normalized_claim'] = df['normalized_claim'].apply(clean_text)
    return df

# Function to clean and deduplicate text
def clean_text_deduplicate(text):
    if pd.isna(text):
        return ""
    # Remove extra whitespace and normalize
    text = ' '.join(text.split())
    # Split into sentences and remove duplicates
    sentences = text.split('. ')
    sentences = [s.strip() for s in sentences if s.strip()]
    unique_sentences = []
    seen = set()
    for s in sentences:
        if s not in seen:
            seen.add(s)
            unique_sentences.append(s)
    # Join back with proper punctuation
    return '. '.join(unique_sentences) + ('.' if text.endswith('.') else '')

# Function to detect language
# Function to detect language (returns True if english, False otherwise)
def detect_language(text):
    if pd.isna(text) or len(text.strip()) < 20:  # Increased threshold for short text
        return True  # Assume english for short or empty texts
    try:
        detected_lang = detect(text)  # Store the result of detect(text)
        return detected_lang == "en"
    except:
        return True  # Fallback to True (assume english) if detection fails


# Function to translate text to english
def translate_to_english(text):
    if pd.isna(text) or text.strip() == "":
        return text
    lang = detect_language(text)
    if lang != 'en':
        try:
            translator = GoogleTranslator(source=lang, target='en')
            translated_text = translator.translate(text)
            return translated_text if translated_text else text
        except Exception as e:
            print(f"Translation failed for text: {text}. Error: {e}")
            return text
    return text

In [15]:
train_data = pd.read_csv("/kaggle/input/check-that-2025-task-2/clef2025-checkthat-lab-main-task2-data/task2/data/train/train-eng.csv") # change path and file name
val_data = pd.read_csv("/kaggle/input/check-that-2025-task-2/clef2025-checkthat-lab-main-task2-data/task2/data/dev/dev-eng.csv")


In [16]:
train_data.shape

(11374, 2)

In [17]:

# Apply preprocessing
train_data = preprocess_data(train_data)

# Apply deduplication and further cleaning to 'post' column
train_data['post'] = train_data['post'].apply(clean_text_deduplicate)

# Apply deduplication and further cleaning to 'normalized_claim' column
train_data['normalized claim'] = train_data['normalized claim'].apply(clean_text_deduplicate)

# Filter out non-english posts with improved detection
# Filter out non-english posts
# Debug: Check how many posts are classified as english
train_data['is_english'] = train_data['post'].apply(lambda x: detect_language(x) if pd.notna(x) else True)
print("Number of posts classified as english:", train_data['is_english'].sum())
print("Sample of non-english posts (first 5):")
non_english_samples = train_data[~train_data['is_english']].head(5)
print(non_english_samples[['post']])

# Filter out non-english posts
train_data = train_data[train_data['is_english']]

# Remove the temporary 'is_english' column
train_data = train_data.drop(columns=['is_english'])

# Translate non-english posts to english
# train_data['post'] = train_data['post'].apply(translate_to_english)

Number of posts classified as english: 10658
Sample of non-english posts (first 5):
                                                 post
3   Pa alam sayu idol🥺 Pa alam sayu idol🥺 Pa alam ...
15  वीडियो को ज्यादा से ज्यादा शेयर करो जय श्री रा...
35  गाय भारत की माता है, ऑस्ट्रेलिया में देखों कौन...
63  Shhhhhhhhhh! Shhhhhhhhhh! Shhhhhhhhhh! THE FED...
85  روسی صدر پیوٹن کا قوم سے اہم خطاب میں پاکستان ...


In [18]:

# Remove rows where both 'post' and 'normalized claim' are empty
train_data = train_data[(train_data['post'] != '') | (train_data['normalized claim'] != '')]

In [19]:
train_data.shape

(10658, 2)

In [20]:
train_data

,post,normalized claim
0,Lieutenant Retired General Asif Mumtaz appoint...,Pakistani government appoints former army gene...
1,A priceless clip of 1970 of Bruce Lee playing ...,Late actor and martial artist Bruce Lee playin...
2,Hydrate YOURSELF W After Waking Up Water 30 mi...,Drinking water at specific times can have diff...
4,Look how the media LIE TO STIR UP TROUBLE So a...,Kendall Jenner doctored a photo of her holding...
5,🚨 fact check 🚨Pfizer was not part of Donald Tr...,Pfizer is not part of Operation Warp Speed
...,...,...
11369,"Here in Bengal, an election rally of the Bhara...","Here in Bengal, an election rally of the Bhara..."
11370,We urge employers to show flexibility to emplo...,Employers cannot penalise workers who are foll...
11371,An ad being run in Georgia by Stephen Millerâ€...,“Kamala Harris said disaster aid should go to ...
11372,Joe Biden takes a knee after spotting an Ameri...,The claim: Presidential candidate Joe Biden kn...


In [21]:
train_data = train_data.sample(frac=1).reset_index(drop=True)
val_data = val_data.sample(frac=1).reset_index(drop=True)

ds = DatasetDict({
        'train': Dataset.from_pandas(train_data),
        'validation': Dataset.from_pandas(val_data)
        })

In [22]:
train_data.shape


(10658, 2)

In [23]:
val_data.shape

(1171, 2)

In [24]:
# Define tokenization function
def tokenize_sample_data(data):
    # Max token size is set to 512 (reduced from 1024) and 128 for inputs and labels
    input_feature = tokenizer(
        data["post"], truncation=True, max_length=512, padding="max_length"
    )
    label = tokenizer(
        data["normalized claim"], truncation=True, max_length=128, padding="max_length"
    )
    # Replace pad_token_id with -100 in labels to ignore padding in loss
    labels = [l if l != tokenizer.pad_token_id else -100 for l in label["input_ids"]]
    return {
        "input_ids": input_feature["input_ids"],
        "attention_mask": input_feature["attention_mask"],
        "labels": labels,
    }

In [25]:
# Apply tokenization to datasets
ds = ds.map(tokenize_sample_data, batched=False)  # batched=False since the function expects single samples

# Assign tokenized datasets
train_dataset = ds['train']
eval_dataset = ds['validation']

Map:   0%|          | 0/10658 [00:00<?, ? examples/s]

Map:   0%|          | 0/1171 [00:00<?, ? examples/s]

In [26]:
# def tokenize_sample_data(data):
#     # Max token size is set to 1024 and 128 for inputs and labels, respectively.
#     input_feature = tokenizer(data["post"], truncation=True, max_length=1024)
#     label = tokenizer(data["normalized claim"], truncation=True, max_length=128)
#     return {
#         "input_ids": input_feature["input_ids"],
#         "attention_mask": input_feature["attention_mask"],
#         "labels": label["input_ids"],
#     }


# tokenized_ds = ds.map(
#     tokenize_sample_data,
#     remove_columns=["post", "normalized claim"],
#     batched=True,
#     batch_size=1)


def tokenize_sentence(arg):
    encoded_arg = tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    preds, labels = eval_arg
    # Replace -100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Convert id tokens to text
    text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for scoring
    text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
    sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
    # compute METEOR score with custom tokenization
    return metric.compute(
        predictions=text_preds,
        references=text_labels
        )


## Training

In [27]:
training_args = Seq2SeqTrainingArguments(
    output_dir = f"saved-models-{model_code}",
    num_train_epochs = 8,  # epochs
    learning_rate = 5e-4,
    lr_scheduler_type = "linear",
    warmup_steps = 90,
    optim = "adamw_torch",  #adam_hf was not a valid optimizer
    weight_decay = 0.01,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 4,
    eval_steps = 100,
    fp16=True,
    predict_with_generate=True,
    generation_max_length = 128,
    logging_steps = 10,
    push_to_hub = False,
    save_strategy="steps",
    save_total_limit=2,         
)



trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer = tokenizer
)

/tmp/ipykernel_31/1275401652.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [28]:
# trainer.evaluate()

In [29]:
# import gc
# import torch

# # After evaluation, clear memory before training
# trainer.model.cpu()  # Move model temporarily to CPU
# del trainer
# gc.collect()
# torch.cuda.empty_cache()

In [30]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, config=config).to(device)

In [ ]:
trainer.train()

os.makedirs(f"{model_code}/finetuned_{model_code}", exist_ok=True)

if hasattr(trainer.model, "module"):
    trainer.model.module.save_pretrained(f"./{model_code}/finetuned_{model_code}")
else:
    trainer.model.save_pretrained(f"./{model_code}/finetuned_{model_code}")

print("Training done")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
10,3.352500
20,3.188700
30,3.280400
40,2.883100
50,2.906500
60,2.803600
70,2.707700
80,2.766100
90,2.520300
100,2.957400


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'num_beams': 15, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors wer

In [ ]:
# loss= 0.524 

In [ ]:
import matplotlib.pyplot as plt

# Extract loss from the trainer's log history
logs = trainer.state.log_history
loss_values = [log["loss"] for log in logs if "loss" in log]
steps = [log["step"] for log in logs if "loss" in log]

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(steps, loss_values, label="Training Loss", color="blue")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend()
plt.show()

In [ ]:

# Save the plot to a file
plt.savefig("Training loss_curve.png", bbox_inches='tight', dpi=300)  # You can use .png, .jpg, .pdf, etc.


In [ ]:
# Create save directory if it doesn't exist
save_path = f"./{model_code}/finetuned_{model_code}"
os.makedirs(save_path, exist_ok=True)

# ✅ Save the tokenizer
tokenizer.save_pretrained(save_path)

print(f"✅ Model and tokenizer saved to {save_path}")

In [ ]:
import gc
import torch

del trainer
del model  # if declared separately
gc.collect()
torch.cuda.empty_cache()


In [ ]:
# from huggingface_hub.utils import validate_hf_hub_args
# validate_hf_hub_args.repo_id = None  # clear repo_id forcibly if it's stored somewhere
# # 

In [ ]:
!rm -rf /kaggle/working/saved-models-pegasus-xsum/checkpoint-5500


In [ ]:
# # Zip a folder
# !zip -r finetuned_pegasus_xsum.zip /kaggle/working/pegasus-xsum/finetuned_pegasus-xsum



In [ ]:
# # Generate download link
# from IPython.display import FileLink
# FileLink(f"/kaggle/working/finetuned_pegasus_xsum.zip")


In [ ]:
# import os
# dataset_name = "finetuned-pegasus-xsum-output"
# os.makedirs("my_dataset", exist_ok=True)

# # Move zip into dataset folder
# !cp /kaggle/working/ffinetuned_pegasus_xsum.zip my_dataset/

# # Create metadata
# metadata = {
#     "title": dataset_name,
#     "id": "atherhashmi/finetuned-pegasus-xsum-output",
#     "licenses": [{"name": "CC0-1.0"}]
# }
# import json
# with open("my_dataset/dataset-metadata.json", "w") as f:
#     json.dump(metadata, f)


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Path to your saved model
model_path = f"/kaggle/working/{model_code}/finetuned_{model_code}"

# Load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
print(model)

In [ ]:
eval_args = Seq2SeqTrainingArguments(
    output_dir=save_path,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    generation_max_length=128,
    logging_steps=10  # optional
)

In [ ]:
from transformers import Seq2SeqTrainer

eval_trainer = Seq2SeqTrainer(
    model=model,
    args=eval_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=metrics_func,
    eval_dataset=eval_dataset
)


In [ ]:
eval_trainer.evaluate()

In [ ]:
# Inference function with prompting
def generate_normalized_claim(post):
    prompt = f"### Instruction: Summarize the following text by identifying the central assertion in a concise, factual statement:\n\n{post}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_new_tokens=120,
            num_beams=5,
            temperature=0.6,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = prediction.split("### Response:")[-1].strip()
    torch.cuda.empty_cache()
    return response

In [ ]:
from tqdm import tqdm

In [ ]:
#test_df = preprocess_data(test_df)
# test_df = test_df[test_df['language'].isin(valid_languages)]  # Filter for valid languages

# # Generate predictions for test data
dev_predictions = []
for post in tqdm(val_dataset['post'], desc="Generating predictions",unit="post"):
    pred = generate_normalized_claim(post)
    dev_predictions.append(pred)

# # Save test predictions
val_dataset['normalized claim'] = dev_predictions
test_df.to_csv("task2_dev_eng.csv",index=False)
print("dev predictions saved to dev_predictions.csv")
print("Sample test predictions:")
for i in range(min(5, len(test_df))):
    print(f"Post: {val_dataset['post'].iloc[i]}")
    print(f"Predicted Normalized Claim: {val_dataset['normalized claim'].iloc[i]}\n")

In [ ]:
test_df = pd.read_csv("/kaggle/input/check-that-2025-task-2/clef2025-checkthat-lab-main-task2-data/task2/data/test/test-eng.csv")
#test_df = preprocess_data(test_df)
# test_df = test_df[test_df['language'].isin(valid_languages)]  # Filter for valid languages

# # Generate predictions for test data
test_predictions = []
for post in tqdm(test_df['post'], desc="Generating predictions",unit="post"):
    pred = generate_normalized_claim(post)
    test_predictions.append(pred)

# # Save test predictions
test_df['normalized claim'] = test_predictions
test_df.to_csv("task2_eng.csv",index=False)
print("Test predictions saved to test_predictions.csv")
print("Sample test predictions:")
for i in range(min(5, len(test_df))):
    print(f"Post: {test_df['post'].iloc[i]}")
    print(f"Predicted Normalized Claim: {test_df['normalized claim'].iloc[i]}\n")

**convert tsv to csv**

In [ ]:
# import csv

# # Change this path to where your file is mounted in Kaggle
# tsv_path = "/kaggle/working/task2_mono_english.tsv"
# csv_path = "/kaggle/working/task2_eng.csv"

# # Convert TSV to CSV
# with open(tsv_path, 'r', newline='', encoding='utf-8') as tsv_file, \
#      open(csv_path, 'w', newline='', encoding='utf-8') as csv_file:

#     tsv_reader = csv.reader(tsv_file, delimiter='\t')
#     csv_writer = csv.writer(csv_file, delimiter=',')

#     for row in tsv_reader:
#         csv_writer.writerow(row)

# print(f"CSV file saved at: {csv_path}")


In [ ]:
# import pandas as pd
# df= pd.read_csv("/kaggle/working/task2_eng.csv")

# df.head()

## Inference

In [ ]:
# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(f"./{model_code}/finetuned_{model_code}")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
input_text = "Pence unfollowed Trump, and then changed his banner picture to Biden and Kamala. He’s outta there P...Syrian ."

# Tokenize the Input Text
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    generated_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=5, early_stopping=True)

# Decode the Generated Output
output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# Print the Output
print(f"Generated Output: {output_text}")